In [3]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import pyarrow as pa
import csv
import subprocess

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.1.0 pyspark-shell'

#spark = SparkSession.builder.appName('SparkCassandraApp').master("local[*]").getOrCreate()
import warnings
warnings.filterwarnings('ignore')

In [4]:
KeyspaceName = 'ukrainetweets'
TableName = 'RawTweet'
session.set_keyspace(KeyspaceName)
#session.execute(f'DROP TABLE {TableName}')

In [3]:
import os
import csv
import gzip
from cassandra.cluster import Cluster
import pandas as pd


ReplicationOptions = {'class': 'SimpleStrategy', 'replication_factor': 1}

query = f"CREATE KEYSPACE IF NOT EXISTS {KeyspaceName} WITH REPLICATION = {ReplicationOptions}"
session.execute(query)

TableName = 'RawTweet'
ColumnNames = ["num", "userid","username","acctdesc","location","following","followers","totaltweets","usercreatedts","tweetid","tweetcreatedts","retweetcount","text","hashtags","language","coordinates","favorite_count","is_retweet","original_tweet_id","original_tweet_userid","original_tweet_username","in_reply_to_status_id","in_reply_to_user_id","in_reply_to_screen_name","is_quote_status","quoted_status_id","quoted_status_userid","quoted_status_username","extractedts"]

create_table = f"CREATE TABLE IF NOT EXISTS {KeyspaceName}.{TableName} ({', '.join([f'{column} TEXT' for column in ColumnNames])}, PRIMARY KEY (tweetid))"
session.execute(create_table)

direct = '/home/hduser/Desktop/CA2/CA2'
I = 0
tweets = []
keywords = ['refugee', 'refugees', 'asylum', 'asylum seeker', 'asylum seeking', 'migration', 'refugee crisis']
for file in os.listdir(direct):
    I+=1
    print(I)
    if file.endswith('.csv.gzip'):
        path = os.path.join(direct, file)
        
        with gzip.open(path, 'rt', encoding = 'utf-8') as file:
            reader = csv.reader(file)
            header = next(reader)
            text_index = header.index('text')
            for row in reader:
                if any(keyword in row[text_index].lower() for keyword in keywords):
                    tweets.append(row)
        
df = pd.DataFrame(tweets)
df.head()

df.to_csv('Filtered_Tweets.csv', index = False)

#The cassandra drivers seem to not support the copy command so this was done in the terminal using the following command
'''
COPY RawTweet (num, userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,is_retweet,original_tweet_id,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts)FROM '/home/hduser/Desktop/CA2/Filtered_Tweets.csv'

'''





1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Number of reccords in the cassandra database

In [3]:
count = session.execute('SELECT COUNT(*) FROM RawTweet')
print(count[0])

Row(count=504775)


In [4]:
spark

In [5]:
from pyspark.sql.functions import col
#import pyspark_cassandra
#pyspark --packages com.datastax.spark:spark-cassandra-connector_2.12:3.1.0

#spark.conf.set(s"spark.sql.catalog.cass100", "com.datastax.spark.connector.datasource.CassandraCatalog")
#spark.conf.set(s"spark.sql.catalog.cass100.spark.cassandra.connection.host", "127.0.0.100")

In [58]:
refugee = spark.read.format("org.apache.spark.sql.cassandra").options(table='rawtweet', keyspace=KeyspaceName).load().select("tweetid", "tweetcreatedts", "text", "language", "location")

In [59]:
refugee.show(20)

+-------------------+--------------------+--------------------+--------+--------------------+
|            tweetid|      tweetcreatedts|                text|language|            location|
+-------------------+--------------------+--------------------+--------+--------------------+
|1519862958389547008|2022-04-29 02:15:...|.@UNHCRPoland exp...|      en|                null|
|1506387935939248137| 2022-03-22 21:50:45|Another internati...|      en|              London|
|1507741608120311813|2022-03-26 15:29:...|In the video, Yek...|      en|                null|
|1545516332900356108|2022-07-08 21:13:...|Some of 2 million...|      en|                null|
|1505754403747344385| 2022-03-21 03:53:20|.@aplusk &amp; Mi...|      en|          Behind You|
|1502704923988869130| 2022-03-12 17:55:47|I got my nieces o...|      en| Львів - Mississauga|
|1513017240597286913|2022-04-10 04:53:...|Thank you, Presid...|      en|Middelburg, Zeela...|
|1505428090687741953| 2022-03-20 06:16:40|📅 March 2022

📍...

2023-05-21 23:48:40,139 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=8024891708407858539, "partition key token"=8864834353737856250]' generated server side warning(s): Read 1000 live rows and 8975 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > 8024891708407858539 AND token(tweetid) <= 8864834353737856250 LIMIT 1000; token 8062028009575384022 (see tombstone_warn_threshold)
2023-05-21 23:48:40,155 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=8024891708407858539, "partition key token"=8864834353737856250]' generated server side warning(s):

In [60]:
#remove non english comments
from pyspark.sql.functions import col
refugee = refugee.filter(col('language') == 'en')
refugee.show(20)


+-------------------+--------------------+--------------------+--------+--------------------+
|            tweetid|      tweetcreatedts|                text|language|            location|
+-------------------+--------------------+--------------------+--------+--------------------+
|1536690832757096448|2022-06-14 12:43:...|🇬🇧|#Migration: ...|      en|                null|
|1507045957669687304| 2022-03-24 17:25:30|🙏🏽 #OrlandoBloo...|      en|              Europe|
|1516221156780511237|2022-04-19 01:04:...|.@JoyAnnReid lead...|      en|                null|
|1505778534966517760| 2022-03-21 05:29:13|More than 360.000...|      en|                null|
|1501902545836716034| 2022-03-10 12:47:25|Meet the Kerala c...|      en|           New Delhi|
|1497571486286155776|2022-02-26 13:57:...|@simon_harper @er...|      en|                  UK|
|1589290428704391168| 2022-11-06 16:15:40|⚡️Ukrainian and I...|      en|                null|
|1500986201968234496| 2022-03-08 00:06:12|Request: Helping ...| 

2023-05-21 23:48:40,770 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s): Read 1000 live rows and 8991 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > 7447475080679012599 AND token(tweetid) <= 8024891708407858539 LIMIT 1000; token 7483727941932587962 (see tombstone_warn_threshold)
2023-05-21 23:48:40,783 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s):

In [61]:
from pyspark.sql.functions import lower, col, regexp_replace, concat_ws
from pyspark.sql.types import StringType, FloatType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # for tokenization
nltk.download('wordnet')  # for lemmatization
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hduser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hduser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [62]:
#remove retweets
refugee = refugee.filter(~col('text').rlike('RT @%'))
#make lower case
refugee = refugee.withColumn('text', lower(col('text')))
#removing twitter name
refugee = refugee.withColumn('text', regexp_replace(col('text'), '@[^\s]+', ''))
#removing hashtag
refugee = refugee.withColumn('text', regexp_replace(col('text'), '\\\B#\\S+', ''))
#removing special characters nums and punctuation
refugee = refugee.withColumn('text', regexp_replace(col('text'), '[^A-Za-z0-9 ]+', ''))
#remove single characters
refugee = refugee.withColumn('text', regexp_replace(col('text'), '\\s+[a-zA-Z]\\s+', ''))

#Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    words = nltk.word_tokenize(text)
    lemmatized = ' '.join([lemmatizer.lemmatize(word) for word in words])
    return lemmatized
#make into udf
lemmatize_udf = udf(lemmatize, StringType())
refugee = refugee.withColumn('text', lemmatize_udf('text'))
#Tokenize words, this is done as the stop words remover needs an array of words
tokenizer = Tokenizer(inputCol="text", outputCol="words")
refugee = tokenizer.transform(refugee)
# Removing stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
refugee = remover.transform(refugee)

#recombine the words into text
refugee = refugee.withColumn('text', concat_ws(' ', col("filtered_words")))


In [63]:
refugee.show(20)

2023-05-21 23:48:43,316 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-6784299022377947072, "partition key token"=-5746602870778827712]' generated server side warning(s): Read 1000 live rows and 8804 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > -6784299022377947072 AND token(tweetid) <= -5746602870778827712 LIMIT 1000; token -6747961037251815877 (see tombstone_warn_threshold)
2023-05-21 23:48:43,329 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-6784299022377947072, "partition key token"=-5746602870778827712]' generated server side warn

+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+
|            tweetid|      tweetcreatedts|                text|language|            location|               words|      filtered_words|
+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+
|1506155681962934274| 2022-03-22 06:27:52|amp mila kunis am...|      en|                null|[amp, mila, kunis...|[amp, mila, kunis...|
|1526624766974648330|2022-05-17 18:04:...|people tigray una...|      en|                null|[people, of, tigr...|[people, tigray, ...|
|1570108587254632450| 2022-09-14 17:53:53|southern israelit...|      en|             Atlanta|[the, southern, i...|[southern, israel...|
|1527258513646436353|2022-05-19 12:02:...|jumped action map...|      en|                null|[jumped, into, ac...|[jumped, action, ...|
|1500963723900555264| 2022-03-07 22:36:52|uk res

In [27]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 3.1 MB/s eta 0:00:000:00:01


In [64]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [65]:
def sentiment(text):
    sentiment_analy = SentimentIntensityAnalyzer()
    vader = sentiment_analy.polarity_scores(text)
    return float(vader['compound'])
    

In [66]:
udfSentiment = udf(sentiment, FloatType())
refugee = refugee.withColumn('sentiment polarity', udfSentiment('text'))
refugee.persist()
refugee.show(20)


2023-05-21 23:51:15,023 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s): Read 1000 live rows and 8991 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > 7447475080679012599 AND token(tweetid) <= 8024891708407858539 LIMIT 1000; token 7483727941932587962 (see tombstone_warn_threshold)
2023-05-21 23:51:15,034 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s):

2023-05-21 23:51:15,960 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s): Read 826 live rows and 7234 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > token(1497267167213015049) AND token(tweetid) <= 8024891708407858539 LIMIT 1000; token 8024818463572463402 (see tombstone_warn_threshold)


+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+
|            tweetid|      tweetcreatedts|                text|language|            location|               words|      filtered_words|sentiment polarity|
+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+
|1536690832757096448|2022-06-14 12:43:...|migration eu inte...|      en|                null|[migration, the, ...|[migration, eu, i...|            0.0772|
|1507045957669687304| 2022-03-24 17:25:30|orlandobloom good...|      en|              Europe|[orlandobloom, go...|[orlandobloom, go...|            0.4019|
|1516221156780511237|2022-04-19 01:04:...|lead thereidout t...|      en|                null|[lead, thereidout...|[lead, thereidout...|           -0.5994|
|1505778534966517760| 2022-03-21 05:29:13|360000 refugee en...|      e

In [68]:
#maker functioin to catagorize sentiment
def sentiment_cat(sentiment):
    if sentiment <= -0.6:
        return "Negative"
    elif -0.6 < sentiment <= -0.2:
        return "Ngative-ish"
    elif -0.2 < sentiment <= 0.2:
        return "Neutral"
    elif 0.2 < sentiment <= 0.6:
        return "Positive-ish"
    else:
        return "Positive"
sentiment_cat_udf = udf(sentiment_cat, StringType())
refugee = refugee.withColumn('Sentiment_Category', sentiment_cat_udf('sentiment polarity'))
refugee.show()


+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+------------------+
|            tweetid|      tweetcreatedts|                text|language|            location|               words|      filtered_words|sentiment polarity|Sentiment_Category|
+-------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+------------------+
|1536690832757096448|2022-06-14 12:43:...|migration eu inte...|      en|                null|[migration, the, ...|[migration, eu, i...|            0.0772|           Neutral|
|1507045957669687304| 2022-03-24 17:25:30|orlandobloom good...|      en|              Europe|[orlandobloom, go...|[orlandobloom, go...|            0.4019|      Positive-ish|
|1516221156780511237|2022-04-19 01:04:...|lead thereidout t...|      en|                null|[lead, thereidout...|[lead, thereidou